Lets work with sample data.

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
# !unzip ../input/diabetic-retinopathy-detection/sample.zip
# !unzip ../input/diabetic-retinopathy-detection/sampleSubmission.csv.zip

In [ ]:
# ! dir sample

In [ ]:
# !unzip ../input/diabetic-retinopathy-detection/trainLabels.csv.zip

In [ ]:
# from PIL import Image

# img = Image.open("./sample/16_right.jpeg")

# import matplotlib.pyplot as plt

# plt.imshow(img)

But there are no labels!

In [ ]:
# import pandas as pd

# pd.read_csv("./trainLabels.csv")

Lets get the first train file

In [ ]:
#!mv ../input/diabetic-retinopathy-detection/train.zip.001 ./train1.zip

In [ ]:
# !unzip train1.zip

# I guess we need to find something else altogether

# New dataset , diabetic-retinopathy-resized

In [ ]:
pd.read_csv("../input/diabetic-retinopathy-resized/trainLabels.csv")['level'].unique()

# classes

In [ ]:
import os
cropped = pd.read_csv("../input/diabetic-retinopathy-resized/trainLabels_cropped.csv")[:5000]

In [ ]:
cropped.iloc[0]

In [ ]:
image_names = os.listdir("../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/")
image_names.sort()
image_names[:5]

In [ ]:
images_path = "../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped"
img_path = os.path.join(images_path, cropped.iloc[3500].image+".jpeg")
img_path

In [ ]:
img = Image.open(img_path)

plt.imshow(img)

img.size

In [ ]:
import torchvision.transforms as transforms
my_transform = transforms.Compose([
    transforms.Resize((299,299)),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
import torch

class retinaDataset(Dataset):
    def __init__(self, imagepath="../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped", total=None,transform=my_transform):
        self.df = pd.read_csv("../input/diabetic-retinopathy-resized/trainLabels_cropped.csv")
        
        if (total is not None):
            self.df = self.df[:total]
        
        self.transform = transform
        
        self.imagepath = imagepath
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.imagepath, self.df.iloc[index].image +".jpeg")
        img = Image.open(img_path)
        
        if(self.transform):
            img = self.transform(img)
        
        return img, torch.tensor(self.df.iloc[index].level)


In [ ]:
train_dataset = retinaDataset(total=5000)

In [ ]:
img, label = train_dataset[0]

plt.imshow(img.permute(1,2,0))
img.shape, label, train_dataset.__len__()

In [ ]:
num_classes = 5
learning_rate = 1e-4
num_epochs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torchvision.models import inception_v3

model = inception_v3(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
model.fc

In [ ]:
model.fc = torch.nn.Linear(in_features=2048, out_features=5, bias=True)

In [ ]:
model.aux_logits = False

In [ ]:
model = model.to(device=device)

In [ ]:
model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
loss_criterion = torch.nn.CrossEntropyLoss()

In [ ]:
batch_size = 32

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
from tqdm import tqdm

for epoch in range(num_epochs):
    for data, target in tqdm(train_dataloader):
        data = data.to(device=device)
        target = target.to(device=device)
        
        score = model(data)
        optimizer.zero_grad()
        
        loss = loss_criterion(score, target)
        loss.backward()
        
        optimizer.step()
    
    print(f"for epoch {epoch}, loss : {loss}")
    

In [ ]:
def check_accuracy(model, loader):
    model.eval()
    
    correct_output = 0
    total_output = 0
    
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device)
            y = y.to(device=device)
            
            score = model(x)
            _,predictions = score.max(1)
            
            correct_output += (y==predictions).sum()
            total_output += predictions.shape[0]
    model.train()
    print(f"out of {total_output} , total correct: {correct_output} with an accuracy of {float(correct_output/total_output)*100}")

In [ ]:
check_accuracy(model, train_dataloader)

In [ ]:
image = Image.open("../input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped/10007_right.jpeg")

img = my_transform(image).unsqueeze(0)

score = model(img)

_, predictions = score.max(1)

plt.imshow(image)

print("predicted: ", predictions)

df = pd.read_csv("../input/diabetic-retinopathy-resized/trainLabels_cropped.csv")

# print(df[image][:5])
# print(df.loc[df[image]=="10_left"].level)
print("actual: ", df.loc[df['image'].isin(['10007_right'])].level)
